In [3]:
import pandas as pd
import numpy as np
import warnings, os
warnings.simplefilter("ignore")

dataframe_sentiment = pd.read_csv("/home/dave/Desktop/code_hackathon/sentiment_result/processed_sentiment.csv")
from datetime import datetime
from dateutil.relativedelta import relativedelta


def add_months(date_int, months):
    # Convert string to datetime
    date_obj = datetime.strptime(str(date_int), "%Y%m%d")
    
    # Add 3 months
    new_date = date_obj + relativedelta(months=months)
    
    # Convert back to string in yyyymmdd format
    new_date_str = new_date.strftime("%Y%m%d")
    
    return int(new_date_str)
stock_df = pd.read_csv("/home/dave/Desktop/code_hackathon/ret_sample.csv")

In [4]:
import os
folder = "/home/dave/Desktop/code_hackathon/merged_data_with_split/"

# Collect all CSV file paths
csv_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".csv")]

# Read and concatenate all CSVs
df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

# Save merged CSV
df.to_csv("/home/dave/Desktop/code_hackathon/merged.csv", index=False)

In [5]:
# clean up empty entries
with open("processed_sentiment.csv", "r") as w:
    lines = w.readlines()
with open("processed_sentiment.csv", "w") as w:
    for line in lines:
        if ",,,,,,,,," not in line:
            w.write(line)

In [6]:
gvkeys = stock_df[["date","gvkey","prc"]][(stock_df["intrinsic_value"]>500) & (stock_df['excntry'] == 'USA')]['gvkey'].unique()
for gvkey in gvkeys:
    if not gvkey in dataframe_sentiment['gvkey'].values:
        continue
    df_a = stock_df[["date","gvkey","prc","ret_1_0"]][(stock_df["intrinsic_value"]>500) & (stock_df["gvkey"]==gvkey)]
    df_b = dataframe_sentiment[dataframe_sentiment["chunks"]>5][(dataframe_sentiment["gvkey"]==gvkey) & (dataframe_sentiment["delta_score"]!=0)]
        # Convert dates to datetime if not already
    df_a['date'] = pd.to_datetime(df_a['date'], format='%Y%m%d')
    df_b['date'] = pd.to_datetime(df_b['date'], format='%Y%m%d')

    # Sort both dataframes by date
    df_a = df_a.sort_values('date')
    df_b = df_b.sort_values('date')

    # Merge with merge_asof to get nearest previous date
    merged_df = pd.merge_asof(
        df_a,
        df_b,
        on='date',
        by='gvkey',
        direction='backward',
        suffixes=('_price', '_sentiment')
    )

    # Filter out rows where the merged date is null (no matching sentiment)
    merged_df = merged_df.dropna(subset=['date'])
    merged_df.to_csv(f"/home/dave/Desktop/code_hackathon/merged_data/merged_{gvkey}.csv", index=False)
    #print(merged_df)

In [83]:
df = pd.read_csv("/home/dave/Desktop/code_hackathon/merged.csv")
df = df.dropna(subset=["delta_score"])
df.to_csv("/home/dave/Desktop/code_hackathon/merged_1.csv", index=False)

In [57]:
df_b

,sentiment,neutral_score,positive_score,negative_score,ticker,gvkey,date,chunks,net_score,delta_score
2802,neutral,0.50704,0.2306,0.2624,AAPL,1690.0,20050201,42,-0.0318,NaN
2803,neutral,0.44417,0.1960,0.3598,AAPL,1690.0,20050504,59,-0.1638,-0.1320
2804,neutral,0.44841,0.1595,0.3921,AAPL,1690.0,20050803,58,-0.2326,-0.0688
2805,negative,0.36834,0.2127,0.4189,AAPL,1690.0,20051201,58,-0.2062,0.0264
2806,neutral,0.45007,0.1443,0.4057,AAPL,1690.0,20060203,52,-0.2614,-0.0552
...,...,...,...,...,...,...,...,...,...,...
2879,neutral,0.74159,0.0116,0.2468,AAPL,1690.0,20240503,11,-0.2352,-0.3337
2881,neutral,0.71613,0.1303,0.1535,AAPL,1690.0,20240802,10,-0.0232,0.2120
2883,neutral,0.90352,0.0085,0.0880,AAPL,1690.0,20241101,10,-0.0795,-0.0563
2885,neutral,0.87780,0.1221,0.0001,AAPL,1690.0,20250131,9,0.1220,0.2015


In [58]:
# Convert dates to datetime if not already
df_a['date'] = pd.to_datetime(df_a['date'], format='%Y%m%d')
df_b['date'] = pd.to_datetime(df_b['date'], format='%Y%m%d')

# Sort both dataframes by date
df_a = df_a.sort_values('date')
df_b = df_b.sort_values('date')

# Merge with merge_asof to get nearest previous date
merged_df = pd.merge_asof(
    df_a,
    df_b,
    on='date',
    by='gvkey',
    direction='backward',
    suffixes=('_price', '_sentiment')
)

# Filter out rows where the merged date is null (no matching sentiment)
merged_df = merged_df.dropna(subset=['date'])

print(merged_df)

          date   gvkey         prc  intrinsic_value sentiment  neutral_score  \
0   2005-02-28  1690.0   76.900002     2.375031e+03   neutral        0.50704   
1   2005-03-31  1690.0   44.860001     2.375031e+03   neutral        0.50704   
2   2005-04-29  1690.0   41.669998     2.375031e+03   neutral        0.50704   
3   2005-05-31  1690.0   36.060001     4.796457e+03   neutral        0.44417   
4   2005-06-30  1690.0   39.759300     4.796457e+03   neutral        0.44417   
..         ...     ...         ...              ...       ...            ...   
239 2025-02-28  1690.0  236.000000     1.549341e+06   neutral        0.87780   
240 2025-03-31  1690.0  241.840000     1.549341e+06   neutral        0.87780   
241 2025-04-30  1690.0  222.130000     1.549341e+06   neutral        0.87780   
242 2025-05-30  1690.0  212.500000     1.469818e+06   neutral        0.69773   
243 2025-06-30  1690.0  200.850000     1.469818e+06   neutral        0.69773   

     positive_score  negative_score tic

In [73]:
merged_df.drop_duplicates(subset=['neutral_score'], keep='first')

,date,gvkey,prc,intrinsic_value,sentiment,neutral_score,positive_score,negative_score,ticker,chunks,net_score,delta_score
0,2025-05-30,323308.0,1.7,685.782173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
merged_df.to_csv('/home/dave/Desktop/code_hackathon/sentiment_result/merged_with_returns_advanced_1.csv', index=False,  float_format="%.5f")

In [13]:
merged_df.sort_values(by='date')


,sentiment,neutral_score,positive_score,negative_score,ticker,gvkey,date,chunks,net_score,previous_net_score,delta,prc,ret_3m,ret_6m,ret_12m
3792,neutral,0.71522,0.08134,0.20344,CNX,120093.0,20050228,75,-0.12210,0.01248,-0.13458,42.189999,0.024887,0.596587,0.727898
6945,neutral,0.72701,0.18055,0.09244,GNTX,5116.0,20050228,11,0.08811,-0.10815,0.19626,33.830002,-0.040497,-0.473249,-0.506355
3953,neutral,0.90610,0.05291,0.04099,CPHD,136804.0,20050228,24,0.01192,0.09375,-0.08183,10.245000,-0.125427,-0.162518,-0.038555
12515,neutral,0.70328,0.27057,0.02616,PEP,8479.0,20050228,89,0.24441,-0.28603,0.53044,53.700001,0.036127,0.015456,0.064804
8720,positive,0.31101,0.68746,0.00153,JOY,13003.0,20050228,10,0.68593,0.24547,0.44046,27.930000,0.212674,0.470462,0.934837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10773,neutral,0.66979,0.10437,0.22585,MRVL,137310.0,20250530,18,-0.12148,0.05297,-0.17445,58.370000,NaN,NaN,NaN
15727,neutral,0.78300,0.08780,0.12919,TGT,3813.0,20250530,16,-0.04139,0.18273,-0.22412,96.700000,NaN,NaN,NaN
6563,neutral,0.75829,0.10840,0.13331,GAP,4990.0,20250530,10,-0.02491,0.00488,-0.02979,21.900000,NaN,NaN,NaN
9386,neutral,0.95539,0.01136,0.03325,LION,28378.0,20250530,66,-0.02189,0.00000,-0.02189,8.880000,NaN,NaN,NaN


In [ ]:
positions = {}


(18059, sentiment              neutral
neutral_score          0.71112
positive_score          0.1113
negative_score         0.17759
ticker                      ZZ
gvkey                   8101.0
date                  20090630
chunks                      30
net_score             -0.06629
previous_net_score     0.14295
delta                 -0.20924
prc                        1.9
ret_3m                0.378947
ret_6m                0.410526
ret_12m               0.710526
Name: 18059, dtype: object)


In [ ]:
best_100_by_year = {}

for year in range(2005, 2019):
    df_year = merged_df[merged_df['date'].astype(str).str.startswith(str(year))]
    if not df_year.empty:
        top_100 = df_year.nlargest(100, 'ret_3m')
        best_100_by_year[year] = top_100[['gvkey', 'ticker', 'date', 'ret_3m', 'sentiment']]

for year, df in best_100_by_year.items():
    print(f"Year: {year}")
    print(df[['gvkey', 'ticker', 'date', 'ret_3m', 'sentiment']])
    print('-' * 40)

In [10]:
usa_prices = stock_df[(stock_df['excntry'] == 'USA') & (stock_df['gvkey'] > 0)][['gvkey', 'prc', 'date']].copy()

usa_prices = usa_prices.sort_values(['gvkey', 'date'])

def calc_forward_return(df, months):
    df = df.copy()
    df['date_forward'] = df['date'].apply(lambda x: add_months(x, months))
    prc_forward = []
    for gvkey, group in df.groupby('gvkey'):
        group = group.sort_values('date')
        dates = group['date'].values
        prices = group['prc'].values
        # For each row, find the first date >= date_forward
        for idx, row in group.iterrows():
            target = row['date_forward']
            # Find index of first date >= target
            i = dates.searchsorted(target)
            if i < len(dates):
                prc_forward.append(prices[i])
            else:
                prc_forward.append(np.nan)
    df['prc_forward'] = prc_forward
    df[f'ret_{months}m'] = (df['prc_forward'] - df['prc']) / df['prc']
    return df

usa_prices_3m = calc_forward_return(usa_prices, 3)
usa_prices_6m = calc_forward_return(usa_prices, 6)
usa_prices_12m = calc_forward_return(usa_prices, 12)

# Example: show returns for 3, 6, and 12 months
usa_prices_returns = usa_prices.copy()
usa_prices_returns['ret_3m'] = usa_prices_3m['ret_3m']
usa_prices_returns['ret_6m'] = usa_prices_6m['ret_6m']
usa_prices_returns['ret_12m'] = usa_prices_12m['ret_12m']

print(usa_prices_returns[['gvkey', 'date', 'prc', 'ret_3m', 'ret_6m', 'ret_12m']].head())

         gvkey      date        prc    ret_3m    ret_6m   ret_12m
18418   1004.0  20050228  11.650000  0.264378  0.542489  1.045494
40928   1004.0  20050331  11.150000  0.438565  0.578475  1.269955
63578   1004.0  20050429  13.600000  0.155147  0.263235  0.961765
86182   1004.0  20050531  14.730000  0.219959  0.081466  0.811270
108819  1004.0  20050630  16.040001  0.097257  0.305486  0.501247


In [ ]:
print(usa_prices_returns[['gvkey', 'date', 'prc', 'ret_3m', 'ret_6m', 'ret_12m']].head(50))

In [ ]:
# Calculate sentiment score as positive_score - negative_score
dataframe_sentiment['sentiment_score'] = dataframe_sentiment['positive_score'] - dataframe_sentiment['negative_score']

# Merge sentiment dataframe with stock returns (3m, 6m, 12m) using gvkey and date
merged_df = pd.merge(
    dataframe_sentiment,
    usa_prices_returns,
    how='inner',
    left_on=['gvkey', 'date'],
    right_on=['gvkey', 'date']
)

# Calculate correlation between sentiment_score and future returns
corr_3m = merged_df['sentiment_score'].corr(merged_df['ret_3m'])
corr_6m = merged_df['sentiment_score'].corr(merged_df['ret_6m'])
corr_12m = merged_df['sentiment_score'].corr(merged_df['ret_12m'])

print("Correlation between sentiment score and 3-month return:", corr_3m)
print("Correlation between sentiment score and 6-month return:", corr_6m)
print("Correlation between sentiment score and 12-month return:", corr_12m)

In [ ]:
# Find the best performing stock by highest 12-month return in merged_df
best_row = merged_df.loc[merged_df['ret_12m'].idxmax()]

# Extract relevant information
best_gvkey = best_row['gvkey']
best_date = best_row['date']
best_sentiment = best_row['sentiment']
best_sentiment_score = best_row['sentiment_score']
best_ticker = best_row['ticker']
best_ret_12m = best_row['ret_12m']

print(f"Best performing stock gvkey: {best_gvkey}, ticker: {best_ticker}, date: {best_date}")
print(f"Initial sentiment assessment: {best_sentiment} (score: {best_sentiment_score})")
print(f"12-month return: {best_ret_12m:.2f}")

In [27]:
best_10_by_year = {}
for year in range(2005, 2019):
    df_year = merged_df[merged_df['date'].astype(str).str.startswith(str(year))]
    if not df_year.empty:
        top_10 = df_year.nlargest(10, 'ret_3m')
        best_10_by_year[year] = top_10[['gvkey', 'ticker', 'date', 'ret_3m', 'sentiment']]

for year, df in best_10_by_year.items():
    print(f"Year: {year}")
    print(df[['gvkey', 'ticker', 'date', 'ret_3m', 'sentiment']])
    print('-' * 40)

Year: 2005
        gvkey ticker        date     ret_3m sentiment
338  147204.0   CCOI  20050331.0  34.400002   neutral
267  109043.0   HOFF  20050331.0  21.782607   neutral
197   30594.0   TRAC  20050331.0   2.606557   neutral
32    61338.0  CVOVQ  20050228.0   1.647458   neutral
225   61984.0    KSW  20050331.0   1.163462   neutral
482    1173.0  AIM.1  20050531.0   1.080201   neutral
318  137130.0   OCNW  20050331.0   1.080000   neutral
592  142945.0  ARD.1  20050729.0   1.066946   neutral
547   62236.0   CPTS  20050729.0   1.053097   neutral
424   22612.0  WTSLQ  20050429.0   0.956897   neutral
----------------------------------------
Year: 2006
         gvkey ticker        date    ret_3m sentiment
1184   64356.0   CIEN  20060831.0  5.476584   neutral
855    15240.0    WWR  20060331.0  3.268293   neutral
1077  120835.0   WSII  20060428.0  1.853968   neutral
901    61077.0    GVP  20060331.0  1.732919   neutral
834    10361.0   SIMC  20060331.0  1.715100   neutral
694    14503.0   IP

In [ ]:
date_cutoff = 20060601

def avg_return(df, name, months):
    count = 0
    total_ret = 0

    for gvkey in df['gvkey'].unique():
        stock_data_df = stock_df[(stock_df['gvkey'] == gvkey) & (stock_df['date'] < date_cutoff) & (stock_df['intrinsic_value'] >= 500)][['prc','date']]
        report_df = df[(df['gvkey']==gvkey) & (df['chunks'] > 10)]

        for report in report_df.itertuples():
            try:
                date = report.date
                index = report.Index
                date_after_three_months = add_months(date,months)
                #print(date, date_after_three_months)
                #print(stock_data_df[stock_data_df['date']>=date].head(1)['prc'].values)
                init_price = float(stock_data_df[stock_data_df['date']>=date].head(1)['prc'].values[0])
                final_price = float(stock_data_df[stock_data_df['date']>=date_after_three_months].head(1)['prc'].values[0])
                ret = (final_price - init_price) / init_price
                weight = report_df.loc[index]['positive_score'] - report_df.loc[index]['negative_score']
                #print(ret*100 * weight)
                count += 1
                total_ret += ret*100 
            except: 
                pass
            
    if count == 0:
        print(f"No data for {name} sentiment report")
    else:        
        print(f"Average return after {name} sentiment report: ", total_ret/count, "based on", count, "reports")


dataframe_sentiment = pd.read_csv(f"/home/dave/Desktop/code_hackathon/merged_1.csv")
for year in range(2005, 2024):
    print(f"Year: {year}")
    positive_results = dataframe_sentiment[(dataframe_sentiment['delta_score']>0) & (dataframe_sentiment['date'].astype(str).str.startswith(str(year)))]
    negative_results = dataframe_sentiment[dataframe_sentiment['delta_score']<0 & (dataframe_sentiment['date'].astype(str).str.startswith(str(year)))]

    avg_return(positive_results, "positive - 0.6 - 6 months", 6)
    avg_return(negative_results, "negative - 0.6 - 6 months", 6)

    avg_return(positive_results, "positive - 0.6 - 6 months", 3)
    avg_return(negative_results, "negative - 0.6 - 6 months", 3)



#avg_return(neutral_results, "neutral")

Year: 2005
Average return after positive - 0.6 - 6 months sentiment report:  5.440964804849186 based on 1350 reports


KeyboardInterrupt: 

In [11]:
df["date"] = pd.to_datetime(df["date"])
df["date"] = df["date"].dt.strftime("%Y%m%d")

In [105]:
df.to_csv("/home/dave/Desktop/code_hackathon/merged_1.csv", index=False)

In [110]:
df = pd.read_csv("/home/dave/Desktop/code_hackathon/merged_1.csv")

In [13]:
df

,date,gvkey,prc,ret_1_0,sentiment,neutral_score,positive_score,negative_score,ticker,chunks,net_score,delta_score,split
0,20050228,8577.0,23.230000,-0.000430,neutral,0.87322,0.0532,0.0735,PNY,31.0,-0.0203,NaN,1.00
1,20050331,8577.0,23.360001,0.005596,neutral,0.89022,0.0538,0.0560,PNY,21.0,-0.0022,0.0181,1.00
2,20050429,8577.0,23.040001,-0.003853,neutral,0.89022,0.0538,0.0560,PNY,21.0,-0.0022,0.0181,1.00
3,20050531,8577.0,22.950001,-0.003906,neutral,0.89022,0.0538,0.0560,PNY,21.0,-0.0022,0.0181,1.00
4,20050630,8577.0,24.459999,0.065795,neutral,0.73414,0.1280,0.1378,PNY,27.0,-0.0098,-0.0076,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306906,20250228,17874.0,232.970000,0.055452,neutral,0.89663,0.0608,0.0425,TMUS,53.0,0.0183,0.0168,1.00
306907,20250331,17874.0,269.690000,0.161394,neutral,0.89663,0.0608,0.0425,TMUS,53.0,0.0183,0.0168,1.00
306908,20250430,17874.0,266.710000,-0.011050,neutral,0.88333,0.0645,0.0522,TMUS,44.0,0.0123,-0.0060,1.00
306909,20250530,17874.0,246.950000,-0.074088,neutral,0.88333,0.0645,0.0522,TMUS,44.0,0.0123,-0.0060,1.01


In [8]:

def backtest_strategy(year, stock_df):
    initial_investment = 10000
    cash_spent = 0
    positions = {}
    total_cash = 0

    signals = df.sort_values('date')
    
    signals = signals[(signals['date'] > int(f"{year}0101")) & (signals['date'] < int(f"{year}0601"))]
    stock_df =  stock_df[(stock_df['date'] > int(f"{year}0101")) & (stock_df['date'] < int(f"{year+1}0601"))]


    for row in signals.itertuples():
        key = (row.gvkey, row.ticker)
        price_row = stock_df[(stock_df['gvkey'] == row.gvkey) & (stock_df['date'] >= row.date)]
        if price_row.empty:
            continue
        price = price_row['prc'].values[0]
        if row.delta_score > 0:
            # Buy $10,000 worth, open new position
            shares = initial_investment / price
            positions.setdefault(key, []).append({'shares': shares, 'buy_price': price, 'buy_date': row.date})
            #print({'key': key, 'buy_price': price, 'buy_date': row.date})
            cash_spent += 10000
        elif row.delta_score <=0 and key in positions:
            # Sell one open positions for this stock
            for pos in positions[key]:
                total_cash += pos['shares'] * price
                #print({'key': key, 'sell_price': price, 'sell_date': row.date})
                break
            positions[key] = []

    # Sell any remaining open positions at their last available price
    #print(positions)
    for key, pos_list in positions.items():
        if pos_list:
            
            # Find last available price for this stock
            last_date = max([p['buy_date'] for p in pos_list])
            last_price_row = stock_df[(stock_df['gvkey'] == key[0]) & (stock_df['date'] >= last_date)].sort_values('date').tail(1)
            if not last_price_row.empty:
                last_price = last_price_row['prc'].values[0]
                for pos in pos_list:
                    total_cash += pos['shares'] * last_price
                    #print({'key': key, 'sell_price': round(float(last_price),2), 'sell_date': row.date})

    final_return = (total_cash - cash_spent) / cash_spent * 100
    print(f"Final overall return {year}: {final_return:.2f}%")
    return final_return

initial_cash =1_000_000
for i in range(2010, 2023):
    initial_cash = initial_cash * float (backtest_strategy(i, stock_df) / 100 + 1)

print("Final cash after all years:", initial_cash, initial_cash/1_000_000 -1)


TypeError: '>' not supported between instances of 'str' and 'int'

In [30]:
def backtest_strategy(year, stock_df):
    initial_investment = 10000
    cash_spent = 0
    positions = {}
    total_cash = 0

    signals = df.sort_values('date')
    signals['date'] = signals['date'].astype(int)
    signals = signals[(signals['date'] > int(f"{year}0101")) & (signals['date'] < int(f"{year+2}0101"))]
    stock_df =  stock_df[(stock_df['date'] > int(f"{year}0101")) & (stock_df['date'] < int(f"{year+2}0301"))]

    # I DID THIS BECAUSE I DON'T KNOW WHICH STOCK GOT DELISTED OR MERGED WITH ANOTHER COMPANY
    # ALL THE STOCKS PURCHASED BEFORE THE END OF THE YEAR WILL BE SOLD BY THE END OF FEBRUARY NEXT YEAR
    

    for row in signals.itertuples():
        key = (row.gvkey, row.ticker)
        price_row = stock_df[(stock_df['gvkey'] == row.gvkey) & (stock_df['date'] >= row.date)]
        if price_row.empty:
            continue
        price = price_row['prc'].values[0]
        split = row.split
        if (split > 1 and round(split,0) > 1):
            for i in positions.get(key, []):
                i['shares'] = i['shares'] * round(split,0)
                #print(f"Adjusted for split {key} on {row.date}: stock split factor {split}")
        elif (split < 1 and round(split,2) < 0.5):
            for i in positions.get(key, []):
                i['shares'] = i['shares'] * round(split,2)
                #print(f"Adjusted for split {key} on {row.date}: stock split factor {split}")
            
        if (row.delta_score > 0.1 ):
            # Buy $10,000 worth, open new position, double down if already owned
            num_of_positions = 0
            if key in positions:
                num_of_positions= len(positions[key])

            #for _ in range(num_of_positions+1):
            shares = initial_investment / price
            positions.setdefault(key, []).append({'shares': shares, 'buy_price': price, 'buy_date': row.date})
            #print({'key': key, 'buy_price': price, 'buy_date': row.date})
            cash_spent += 10000
        elif row.delta_score < 0.1 and key in positions and positions[key] != []:
            # Sell one open positions for this stock
            for pos in positions[key]:
                total_cash += pos['shares'] * price
                positions[key].pop(0)
                #print({'key': key, 'sell_price': price, 'sell_date': row.date})
                break
            if positions[key] == []:
                positions.pop(key)


    # Sell any remaining open positions at their last available price
    #print(positions)

    count = 0
    for key, pos_list in positions.items():
        if pos_list:
            
            # Find last available price for this stock
            last_date = max([p['buy_date'] for p in pos_list])
            last_price_row = stock_df[(stock_df['gvkey'] == key[0]) & (stock_df['date'] >= last_date)].sort_values('date').tail(1)
            if not last_price_row.empty:

                count += 1
                last_price = last_price_row['prc'].values[0]
                last_date = last_price_row['date'].values[0]
                for pos in pos_list:
                    total_cash += pos['shares'] * last_price
                    #print({'key': key, 'sell_price': round(float(last_price),2), 'sell_date': last_date})
    #print(total_cash, cash_spent, cash_spent/10000)
    final_return = (total_cash - cash_spent) / cash_spent * 100
    print(f"Final overall return {year}: {final_return:.2f}%. positions left unsold: {count}")
    return final_return

initial_cash = 1_000_000
for i in range(2010, 2024, 2):
    initial_cash = initial_cash * float (backtest_strategy(i, stock_df) / 100 + 1)

print("Final cash after all years:", initial_cash)


Final overall return 2010: 7.19%. positions left unsold: 370
Final overall return 2012: 14.16%. positions left unsold: 372
Final overall return 2014: -2.50%. positions left unsold: 318
Final overall return 2016: 14.45%. positions left unsold: 357
Final overall return 2018: 4.34%. positions left unsold: 325
Final overall return 2020: 16.17%. positions left unsold: 427
Final overall return 2022: 5.93%. positions left unsold: 437
Final cash after all years: 1753232.9627635751


In [32]:
df['delta_score'].corr(df['ret_1_0'])

np.float64(0.013325294112378932)

In [66]:
def monthly_strategy(month,stock_df):
    initial_investment = 10000
    cash_spent = 0
    total_cash = 0
    count = 0
    signals = df.sort_values('date')
    signals['date'] = signals['date'].astype(int)
    signals = signals[(signals['date'] > int(f"{month}01")) & (signals['date'] < int(f"{month+1}01"))]
    stock_df = stock_df[(stock_df['date'] > int(f"{month}01")) & (stock_df['date'] < int(f"{month+1}01"))]

    for row in signals.itertuples():
        initial_investment = 10000
        if row.delta_score > 0.1:
            initial_investment = initial_investment * (1 + row.ret_1_0)
            if str(initial_investment)=="nan":
                continue
            total_cash += initial_investment
            cash_spent += 10000
            count += 1
            #print(f"Bought {row.gvkey} at {row.prc} on {row.date}, new investment value: {initial_investment} ")
    #print(f"Month: {month}, total cash: {total_cash}, cash spent: {cash_spent}, count: {count}")
    return (total_cash - cash_spent) / cash_spent

start_date = datetime(2010, 1, 1)
end_date = datetime(2024, 12, 30)
monthly_periods = pd.period_range(start=start_date, end=end_date, freq='M')
initial = 1_000_000
for i in list(monthly_periods):
    month = int(i.to_timestamp().to_pydatetime().strftime("%Y%m%d")[0:6])
    initial = initial * float (monthly_strategy(month, stock_df) + 1)
    print(month, monthly_strategy(month, stock_df))
    #break
print("Final cash after all months:", initial)


201001 0.05558843507333306
201002 -0.03723437318486213
201003 0.04809184146186034
201004 0.0766787915404541
201005 0.04617226667777792
201006 -0.07263907434612588
201007 -0.06873262483534183
201008 0.08556589507462772
201009 -0.06266912339170724
201010 0.11521934507947341
201011 0.029520153867879206
201012 0.04343298482738916
201101 0.07909498397402659
201102 0.0177075622238894
201103 0.04499065698852477
201104 0.02316285376018103
201105 0.03283029592364831
201106 -0.0025975245172756593
201107 -0.016125535284538122
201108 -0.035259095384553074
201109 -0.0649006486399174
201110 -0.10176890043721945
201111 0.15274254731377307
201112 0.013596553148171245
201201 -0.00016776861499980442
201202 0.0780639656917087
201203 0.03216809665427108
201204 0.0223728878202643
201205 -0.008574871449392696
201206 -0.06935236056296254
201207 0.02563938543014362
201208 0.00946840473664949
201209 0.03645978144123687
201210 0.022605152187234242
201211 -7.1129682079506665e-06
201212 0.02167132062071014
201301

In [60]:
import numbers

In [50]:
for i in list(monthly_periods):
    print(i.to_timestamp().to_pydatetime().strftime("%Y%m%d")[0:6])

201001
201002
201003
201004
201005
201006
201007
201008
201009
201010
201011
201012
201101
201102
201103
201104
201105
201106
201107
201108
201109
201110
201111
201112
201201
201202
201203
201204
201205
201206
201207
201208
201209
201210
201211
201212
201301
201302
201303
201304
201305
201306
201307
201308
201309
201310
201311
201312
201401
201402
201403
201404
201405
201406
201407
201408
201409
201410
201411
201412
201501
201502
201503
201504
201505
201506
201507
201508
201509
201510
201511
201512
201601
201602
201603
201604
201605
201606
201607
201608
201609
201610
201611
201612
201701
201702
201703
201704
201705
201706
201707
201708
201709
201710
201711
201712
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904
201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111

In [29]:
def backtest_strategy(year, stock_df):
    initial_investment = 10000
    cash_spent = 0
    positions = {}
    total_cash = 0

    signals = df.sort_values('date')
    signals['date'] = signals['date'].astype(int)
    signals = signals[(signals['date'] > int(f"{year}0101")) & (signals['date'] < int(f"{year+12}0101"))]
    stock_df =  stock_df[(stock_df['date'] > int(f"{year}0101")) & (stock_df['date'] < int(f"{year+12}0301"))]

    # I DID THIS BECAUSE I DON'T KNOW WHICH STOCK GOT DELISTED OR MERGED WITH ANOTHER COMPANY
    # ALL THE STOCKS PURCHASED BEFORE THE END OF THE YEAR WILL BE SOLD BY THE END OF FEBRUARY NEXT YEAR
    

    for row in signals.itertuples():
        key = (row.gvkey, row.ticker)
        price_row = stock_df[(stock_df['gvkey'] == row.gvkey) & (stock_df['date'] >= row.date)]
        if price_row.empty:
            continue
        price = price_row['prc'].values[0]
        split = row.split
        if (split > 1 and round(split,0) > 1):
            for i in positions.get(key, []):
                i['shares'] = i['shares'] * round(split,0)
                #print(f"Adjusted for split {key} on {row.date}: stock split factor {split}")
        elif (split < 1 and round(split,2) < 0.5):
            for i in positions.get(key, []):
                i['shares'] = i['shares'] * round(split,2)
                #print(f"Adjusted for split {key} on {row.date}: stock split factor {split}")
            
        if (row.delta_score > 0.1 ):
            # Buy $10,000 worth, open new position, double down if already owned
            num_of_positions = 0
            if key in positions:
                num_of_positions= len(positions[key])

            #for _ in range(num_of_positions+1):
            shares = initial_investment / price
            positions.setdefault(key, []).append({'shares': shares, 'buy_price': price, 'buy_date': row.date})
            #print({'key': key, 'buy_price': price, 'buy_date': row.date})
            cash_spent += 10000
        elif row.delta_score < 0.1 and key in positions and positions[key] != []:
            # Sell one open positions for this stock
            for pos in positions[key]:
                total_cash += pos['shares'] * price
                positions[key].pop(0)
                #print({'key': key, 'sell_price': price, 'sell_date': row.date})
                break
            if positions[key] == []:
                positions.pop(key)


    # Sell any remaining open positions at their last available price
    #print(positions)

    count = 0
    for key, pos_list in positions.items():
        if pos_list:
            
            # Find last available price for this stock
            last_date = max([p['buy_date'] for p in pos_list])
            last_price_row = stock_df[(stock_df['gvkey'] == key[0]) & (stock_df['date'] >= last_date)].sort_values('date').tail(1)
            if not last_price_row.empty:

                count += 1
                last_price = last_price_row['prc'].values[0]
                last_date = last_price_row['date'].values[0]
                for pos in pos_list:
                    total_cash += pos['shares'] * last_price
                    #print({'key': key, 'sell_price': round(float(last_price),2), 'sell_date': last_date})
    #print(total_cash, cash_spent, cash_spent/10000)
    final_return = (total_cash - cash_spent) / cash_spent * 100
    print(f"Final overall return {year}: {final_return:.2f}%. positions left unsold: {count}")
    return final_return

initial_cash = 1_000_000
initial_cash = initial_cash * float (backtest_strategy(2010, stock_df) / 100 + 1)

print("Final cash after all years:", initial_cash)


KeyboardInterrupt: 

In [135]:
def backtest_strategy(year, stock_df):
    initial_investment = 10000
    cash_spent = 0
    positions = {}
    total_cash = 0

    signals = df.sort_values('date')

    signals = signals[(signals['date'] > int(f"{year}0101")) & (signals['date'] < int(f"{year+12}0101"))]
    stock_df =  stock_df[(stock_df['date'] > int(f"{year}0101")) & (stock_df['date'] < int(f"{year+12}0301"))]

    # I DID THIS BECAUSE I DON'T KNOW WHICH STOCK GOT DELISTED OR MERGED WITH ANOTHER COMPANY
    # ALL THE STOCKS PURCHASED BEFORE THE END OF THE YEAR WILL BE SOLD BY THE END OF FEBRUARY NEXT YEAR

    for row in signals.itertuples():
        key = (row.gvkey, row.ticker)
        price_row = stock_df[(stock_df['gvkey'] == row.gvkey) & (stock_df['date'] >= row.date)]
        if price_row.empty:
            continue
        price = price_row['prc'].values[0]
        if (row.delta_score > 0.1 and len(positions)<250) or (row.delta_score > 0 and key in positions):
            # Buy $10,000 worth, open new position
            shares = initial_investment / price
            positions.setdefault(key, []).append({'shares': shares, 'buy_price': price, 'buy_date': row.date})
            #print({'key': key, 'buy_price': price, 'buy_date': row.date})
            cash_spent += 10000
        elif row.delta_score < 0.2 and key in positions and positions[key] != []:
            # Sell all positions for this stock
            for pos in positions[key]:
                total_cash += pos['shares'] * price
            positions.pop(key)
            print(f"Sold all positions for {key} at price {price} on date {row.date}")


    # Sell any remaining open positions at their last available price
    #print(positions)

    count = 0
    for key, pos_list in positions.items():
        if pos_list:
            
            # Find last available price for this stock
            last_date = max([p['buy_date'] for p in pos_list])
            last_price_row = stock_df[(stock_df['gvkey'] == key[0]) & (stock_df['date'] >= last_date)].sort_values('date').tail(1)
            if not last_price_row.empty:

                count += 1
                last_price = last_price_row['prc'].values[0]
                last_date = last_price_row['date'].values[0]
                for pos in pos_list:
                    total_cash += pos['shares'] * last_price
                    #print({'key': key, 'sell_price': round(float(last_price),2), 'sell_date': last_date})

    final_return = (total_cash - cash_spent) / cash_spent * 100
    print(f"Final overall return {year}: {final_return:.2f}%. positions left unsold: {count}")
    return final_return

initial_cash = 1_000_000
initial_cash = initial_cash * float (backtest_strategy(2010, stock_df) / 100 + 1)

print("Final cash after all years:", initial_cash)


Sold all positions for (180402.0, 'HSNI') at price 21.659999847 on date 20100331
Sold all positions for (150278.0, 'CRI') at price 28.659999847 on date 20100331
Sold all positions for (164680.0, 'KS') at price 9.1599998474 on date 20100331
Sold all positions for (7977.0, 'XEL') at price 21.200000763 on date 20100430
Sold all positions for (177376.0, 'DFS') at price 14.899999619 on date 20100430
Sold all positions for (25338.0, 'BBBYQ') at price 43.759998322 on date 20100430
Sold all positions for (7186.0, 'MHS') at price 64.559997559 on date 20100430
Sold all positions for (31224.0, 'ESINQ') at price 112.48000336 on date 20100430
Sold all positions for (4560.0, 'FDO') at price 36.61000061 on date 20100430
Sold all positions for (14960.0, 'LH') at price 75.709999084 on date 20100430
Sold all positions for (1920.0, 'AVP') at price 33.869998932 on date 20100430
Sold all positions for (2044.0, 'BCR') at price 86.620002747 on date 20100430
Sold all positions for (5229.0, 'GR') at price 70.5

In [163]:
def backtest_mean_reversion_strategy(
    year: int,
    price_df: pd.DataFrame,
    start_cash: float,
    portfolio_pct_per_trade: float,
    rsi_buy_threshold: float,
    rsi_sell_threshold: float,
    stop_loss_pct: float,
    take_profit_pct: float
):
    """
    Backtests a mean-reversion strategy with RSI, stop-loss, and take-profit.
    (Robust version that only requires 'gvkey', 'date', and 'prc' columns)
    """
    cash = start_cash
    # Positions structure: {gvkey: {'shares': float, 'entry_price': float}}
    # We now use gvkey as the sole key for simplicity and robustness.
    positions = {}
    trade_count = 0

    # Filter data for the relevant period
    df = price_df[(price_df['date'] >= int(f"{year}0101")) & (price_df['date'] < int(f"{year+1}0101"))].copy()
    
    # Pre-calculate RSI for each stock
    df['rsi'] = df.groupby('gvkey')['prc'].transform(calculate_rsi)

    # Get all unique trading days and iterate day by day
    trading_days = sorted(df['date'].unique())

    for day in trading_days:
        # --- 1. CHECK FOR EXITS (Highest Priority) ---
        # Get prices for today to check for exits
        today_prices = df[df['date'] == day].set_index('gvkey')
        
        positions_to_close = []
        # We iterate directly over the gvkey of our open positions
        for gvkey, pos in positions.items():
            if gvkey not in today_prices.index:
                continue # No price data for this stock today

            current_price = today_prices.loc[gvkey, 'prc']
            entry_price = pos['entry_price']
            
            # Check Stop-Loss
            if current_price <= entry_price * (1 - stop_loss_pct):
                positions_to_close.append(gvkey)
            # Check Take-Profit
            elif current_price >= entry_price * (1 + take_profit_pct):
                positions_to_close.append(gvkey)
            # Check RSI Sell Signal
            elif today_prices.loc[gvkey, 'rsi'] > rsi_sell_threshold:
                positions_to_close.append(gvkey)

        # Execute the exits
        for gvkey in positions_to_close:
            if gvkey in positions:
                current_price = today_prices.loc[gvkey, 'prc']
                shares = positions[gvkey]['shares']
                cash += shares * current_price
                trade_count += 1
                del positions[gvkey]

        # --- 2. CHECK FOR ENTRIES ---
        if cash > 0:
            # Find stocks with a new RSI buy signal
            yesterday = day - 1
            if yesterday in trading_days:
                # Get today's data (no longer need 'ticker')
                today_data = df[df['date'] == day][['gvkey', 'prc', 'rsi']]
                # Get yesterday's RSI data
                yesterday_rsi = df[df['date'] == yesterday][['gvkey', 'rsi']].rename(columns={'rsi': 'rsi_prev'})
                # Merge explicitly on gvkey
                merged_data = pd.merge(today_data, yesterday_rsi, on='gvkey', how='left')
                
                # Condition: RSI today < 30 AND RSI yesterday >= 30
                buy_signals = merged_data[
                    (merged_data['rsi'] < rsi_buy_threshold) & 
                    (merged_data['rsi_prev'] >= rsi_buy_threshold)
                ]

                for _, row in buy_signals.iterrows():
                    # Use row['gvkey'] as the key
                    gvkey = row['gvkey']
                    if gvkey not in positions: # Don't open a new position if one already exists
                        current_price = row['prc']
                        position_value = cash * portfolio_pct_per_trade
                        shares_to_buy = position_value / current_price
                        
                        positions[gvkey] = {'shares': shares_to_buy, 'entry_price': current_price}
                        cash -= position_value
                        trade_count += 1
                        
                        if cash <= 0: # Stop buying if out of cash
                            break

    # --- FINAL LIQUIDATION ---
    # Sell any remaining positions at the last price of the year
    last_day = trading_days[-1]
    final_prices = df[df['date'] == last_day].set_index('gvkey')
    
    for gvkey, pos in list(positions.items()):
        if gvkey in final_prices.index:
            current_price = final_prices.loc[gvkey, 'prc']
            cash += pos['shares'] * current_price
            trade_count += 1
            del positions[gvkey]

    final_portfolio_value = cash
    final_return_pct = ((final_portfolio_value - start_cash) / start_cash) * 100 if start_cash > 0 else 0

    return {
        'year': year,
        'final_return_pct': final_return_pct,
        'final_value': final_portfolio_value,
        'trades': trade_count,
        'start_value': start_cash
    }

In [164]:
# --- MAIN EXECUTION LOOP ---

# Strategy Parameters
STARTING_CASH = 100_000
PORTFOLIO_PCT_PER_TRADE = 0.05  # 5% of portfolio per trade
RSI_BUY_THRESHOLD = 30
RSI_SELL_THRESHOLD = 70
STOP_LOSS_PCT = 0.10  # 10%
TAKE_PROFIT_PCT = 0.20 # 20%

current_portfolio_value = STARTING_CASH
all_results = []

print(f"Starting Mean Reversion Backtest with ${STARTING_CASH:,.2f}\n")
print("-" * 80)
print(f"{'Year':<6} | {'Start Value':>15} | {'End Value':>15} | {'Return %':>11} | {'Trades':>7}")
print("-" * 80)

for year in range(2010, 2024):
    result = backtest_mean_reversion_strategy(
        year=year,
        price_df=stock_df,
        start_cash=current_portfolio_value,
        portfolio_pct_per_trade=PORTFOLIO_PCT_PER_TRADE,
        rsi_buy_threshold=RSI_BUY_THRESHOLD,
        rsi_sell_threshold=RSI_SELL_THRESHOLD,
        stop_loss_pct=STOP_LOSS_PCT,
        take_profit_pct=TAKE_PROFIT_PCT
    )
    
    all_results.append(result)
    current_portfolio_value = result['final_value']
    
    print(f"{result['year']:<6} | ${result['start_value']:>13,.2f} | ${result['final_value']:>13,.2f} | {result['final_return_pct']:>10.2f}% | {result['trades']:>7}")

print("-" * 80)
print(f"\nFinal portfolio value after all years: ${current_portfolio_value:,.2f}")
total_return = ((current_portfolio_value - STARTING_CASH) / STARTING_CASH) * 100
print(f"Total cumulative return: {total_return:.2f}%")

Starting Mean Reversion Backtest with $100,000.00

--------------------------------------------------------------------------------
Year   |     Start Value |       End Value |    Return % |  Trades
--------------------------------------------------------------------------------
2010   | $   100,000.00 | $   100,000.00 |       0.00% |       2
2011   | $   100,000.00 | $   100,000.00 |       0.00% |       0
2012   | $   100,000.00 | $   100,000.00 |       0.00% |       0


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [95]:
df['date'][1].year

AttributeError: 'str' object has no attribute 'year'

In [36]:
float (backtest_strategy(i, stock_df) / 100 + 1)

Final overall return 2023: 13.98%


1.139783684138995

In [36]:
positions

{(7486.0,
  'MOD'): [{'shares': np.float64(167.50418546389125),
   'buy_price': np.float64(59.700000763),
   'buy_date': 20240131}, {'shares': np.float64(99.81035788464644),
   'buy_price': np.float64(100.19000244),
   'buy_date': 20240731}],
 (8699.0,
  'POWL'): [{'shares': np.float64(113.1221699929363),
   'buy_price': np.float64(88.400001526),
   'buy_date': 20240131}, {'shares': np.float64(69.73500993991325),
   'buy_price': np.float64(143.3999939),
   'buy_date': 20240731}],
 (8850.0,
  'KWR'): [{'shares': np.float64(52.6482040198925),
   'buy_price': np.float64(189.94000244),
   'buy_date': 20240229}],
 (22583.0,
  'GMS'): [{'shares': np.float64(118.82128760547107),
   'buy_price': np.float64(84.160003662),
   'buy_date': 20240229}],
 (170765.0,
  'ANDAU.1'): [{'shares': np.float64(217.3440635666885),
   'buy_price': np.float64(46.009998322),
   'buy_date': 20240229}],
 (39157.0,
  'AVPT'): [{'shares': np.float64(1298.7013308652395),
   'buy_price': np.float64(7.6999998093),
   '

In [ ]:
l = []
for i in positive_results.itertuples():
    l.append(i.positive_score-i.negative_score)
l.sort(reverse=True)
print(l)
    

In [ ]:
positive_results[(positive_results['positive_score']-positive_results['negative_score'])>0.6]

In [ ]:
print(len(positive_results), len(negative_results), len(neutral_results))

In [ ]:
count = 0
total_ret = 0

for gvkey in negative_results['gvkey'].unique():
    stock_data_df = stock_df[(stock_df['gvkey'] == gvkey) & (stock_df['date'] < 20060601)][['prc','date']]
    report_df = negative_results[negative_results['gvkey']==gvkey]
    for date in list(report_df['date']):
        try:
            date_after_three_months = add_three_months(date)
            #print(date, date_after_three_months)
            init_price = float(stock_data_df[stock_data_df['date']>=date].head(1)['prc'].values[0])
            final_price = float(stock_data_df[stock_data_df['date']>=date_after_three_months].head(1)['prc'].values[0])
            ret = (final_price - init_price) / init_price
            print(ret*100)
            count += 1
            total_ret += ret*100
        
        except:
            continue
print("Average return after negative sentiment report: ", total_ret/count)

In [ ]:
data = stock_df[(stock_df['gvkey'] == 25338) & (stock_df['date'] < 20060601)][['prc','date']]

In [ ]:
positive_results[positive_results['gvkey']==25338]

In [ ]:
bbbyq = positive_results[positive_results['gvkey']==25338]

In [ ]:
list(bbbyq['date'])

In [ ]:
for date in list(bbbyq['date']):
    date_after_three_months = add_three_months(date)
    print(date, date_after_three_months)
    init_price = float(data[data['date']>=date].head(1)['prc'].values[0])
    final_price = float(data[data['date']>=date_after_three_months].head(1)['prc'].values[0])
    ret = (final_price - init_price) / init_price
    print(ret*100)

In [ ]:
for date in list(bbbyq['date']):
    date_after_three_months = add_three_months(date)
    print(date, date_after_three_months)
    print(float(data[data['date']>=date].head(1)['prc'].values[0]))
    print(float(data[data['date']>=date_after_three_months].head(1)['prc'].values[0]))
